In [1]:
"""libs"""
import time
import datetime
import re
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import sqlite3
import multiprocessing

br_conn = sqlite3.connect("./database/br_base_cnpj.db")
cadastrur_file = ("./database/CADASTUR_SIGA_SIMPLES_APPLE.xlsx")
# obteendo o caminho do diretório atual e construindo o caminho do arquivo a partir dele
current_dir = os.getcwd()

In [2]:
# lendo os dados do cadastrur
cadastrur_dados = pd.read_excel(cadastrur_file, dtype='string')

In [3]:
cadastrur_dados.rename(columns={'NúmerodeInscriçãodoCNPJ':'CNPJ', 'NomedaPessoaJurídica':"RAZAO_SOCIAL",
       'NomeFantasia':'NOME_FANTASIA', 'SituaçãoCadastral':'SITUACAO_CADASTRAL', 'SituaçãodaAtividade':'SITUACAO_ATIVIDADE',
       'TipodeEstabelecimento':"TIPO_ESTABELECIMENTO", 'NaturezaJurídica':'NATUREZA_JURIDICA', 'Porte':'PORTE',
       'EndereçoCompletoReceitaFederal':'ENDERECO_RFB', 'E - mailComercial':'EMAIL', 'Website':'SITE',
       'NúmerodoCertificado':'NUMERO_CERTIFICADO', 'ValidadedoCertificado':'VALIDADE_CERTIFICADO', 'Tipo':'TIPO', 'Especialidade':'ESPECIALIDADE',
       'Idiomas':'IDIOMAS', 'TelefoneComercial':'TELEFONE_COMERCIAL', 'Município':'MUNICIPIO',
       'EndereçoCompletoComercial':'ENDERECO_COMERCIAL', 'CNAE(S)relacionadosàatividade':'CNAE_ATIVIDADE',
       'StatusAssociadosAbrasel':'TIPO_ASSOCIADO'}, inplace=True)
cadastrur_dados = cadastrur_dados[["CNPJ", "RAZAO_SOCIAL", "NOME_FANTASIA", 
                                   "SITUACAO_CADASTRAL", "SITUACAO_ATIVIDADE", "TIPO_ESTABELECIMENTO", 
                                   "NATUREZA_JURIDICA", "PORTE", "ENDERECO_RFB", "EMAIL", "SITE", 
                                   "NUMERO_CERTIFICADO", "VALIDADE_CERTIFICADO",  "TIPO", "ESPECIALIDADE", 
                                   "IDIOMAS", "UF", "TELEFONE_COMERCIAL", "MUNICIPIO","ENDERECO_COMERCIAL", 
                                   "CNAE_ATIVIDADE", "TIPO_ASSOCIADO"]]
cadastrur_dados.head(1)

,CNPJ,RAZAO_SOCIAL,NOME_FANTASIA,SITUACAO_CADASTRAL,SITUACAO_ATIVIDADE,TIPO_ESTABELECIMENTO,NATUREZA_JURIDICA,PORTE,ENDERECO_RFB,EMAIL,...,VALIDADE_CERTIFICADO,TIPO,ESPECIALIDADE,IDIOMAS,UF,TELEFONE_COMERCIAL,MUNICIPIO,ENDERECO_COMERCIAL,CNAE_ATIVIDADE,TIPO_ASSOCIADO
0,00649660000106,LUIZA DE JESUS LEME BAURU,LUIZA DE JESUS LEME BAURU ME,Regular,Operação,Matriz,Empresário (Individual),MICROEMPRESA,BRAZ FERNANDES Bauru Parque Real CEP: 1705702...,ESCRITAFISCAL@ESCRITORIOTIMONEIRO.COM.BR,...,30/03/2024,Restaurante,Brasileira,Português,SP,(14)3239-9388,Bauru,Vista Alegre Bauru Parque Vista Alegre CEP: 1...,5611203,LEAD


In [4]:
# lendo nossa base da receita
receita_dados = pd.read_sql_query("SELECT * FROM estabelecimentos", br_conn, dtype='string')

In [11]:
# concatenando os dois dataframes
cadastrur_dados['CNPJ'] = cadastrur_dados['CNPJ'].astype('str')
receita_dados['CNPJ'] = receita_dados['CNPJ'].astype('str')
merge_dados_dados = cadastrur_dados.merge(receita_dados, how='left', on='CNPJ')

In [12]:
merge_dados_dados.columns

Index(['CNPJ', 'RAZAO_SOCIAL_x', 'NOME_FANTASIA_x', 'SITUACAO_CADASTRAL_x',
       'SITUACAO_ATIVIDADE', 'TIPO_ESTABELECIMENTO', 'NATUREZA_JURIDICA',
       'PORTE', 'ENDERECO_RFB', 'EMAIL', 'SITE_x', 'NUMERO_CERTIFICADO',
       'VALIDADE_CERTIFICADO', 'TIPO', 'ESPECIALIDADE', 'IDIOMAS', 'UF',
       'TELEFONE_COMERCIAL', 'MUNICIPIO', 'ENDERECO_COMERCIAL',
       'CNAE_ATIVIDADE', 'TIPO_ASSOCIADO', 'RAZAO_SOCIAL_y', 'NOME_FANTASIA_y',
       'RUA', 'NUMERO', 'COMPLEMENTO', 'BAIRRO', 'CIDADE', 'ESTADO', 'CEP',
       'LATITUDE', 'LONGITUDE', 'TELEFONE1', 'SITE_y', 'CNAE_DESCRICAO',
       'HORARIO_FUNCIONAMENTO', 'INSTAGRAM', 'FACEBOOK', 'OPCOES_DE_SERVICO',
       'SITUACAO_CADASTRAL_y', 'DATA_SITUACAO_CADASTRAL'],
      dtype='object')

In [25]:
print(len(merge_dados_dados['CNPJ']))
merge_dados_dados.dropna(subset=['LATITUDE', 'LONGITUDE'],inplace=True)
merge_dados_dados['INSTAGRAM'] = merge_dados_dados['INSTAGRAM'].str.replace('\*', '', regex=True)
merge_dados_dados['INSTAGRAM'] = merge_dados_dados['INSTAGRAM'].str.replace('.', '', regex=True)
merge_dados_dados = merge_dados_dados[merge_dados_dados['INSTAGRAM'] != "@"]
merge_dados_dados.rename(columns={"RAZAO_SOCIAL_x":"RAZAO_SOCIAL", 'NOME_FANTASIA_x':'NOME_FANTASIA', 'SITUACAO_CADASTRAL_x':'SITUACAO_CADASTRAL'}, inplace=True)
merge_dados_dados = merge_dados_dados[['CNPJ', 'RAZAO_SOCIAL', 'NOME_FANTASIA', 'SITUACAO_CADASTRAL',
       'SITUACAO_ATIVIDADE', 'TIPO_ESTABELECIMENTO', 'SITE_x', 'NUMERO_CERTIFICADO', 
       'TIPO', 'ESPECIALIDADE', 'IDIOMAS', 'UF','TELEFONE_COMERCIAL',
       'CNAE_ATIVIDADE', 'TIPO_ASSOCIADO', 'ENDERECO_COMERCIAL', 'RUA', 'NUMERO', 'COMPLEMENTO', 'BAIRRO', 'CIDADE','CEP',
       'LATITUDE', 'LONGITUDE','SITE_y', 'CNAE_DESCRICAO','HORARIO_FUNCIONAMENTO', 'INSTAGRAM', 'FACEBOOK', 'OPCOES_DE_SERVICO']]
print(len(merge_dados_dados['CNPJ']))


7283
7283


In [28]:
merge_dados_dados.to_excel('merge_data_cadastrur-rfb.xlsx', sheet_name="CADASTRUR RFB", index=False)

In [3]:
urls = []
for nome_fantasia, uf, cidade, bairro in merge_dados_dados[["NOME_FANTASIA", "UF", "CIDADE" ,"BAIRRO"]].itertuples(index=False):
    urls.append(f"https://www.google.com/maps/search/{nome_fantasia}+{uf}+{cidade}+{bairro}".replace(' ', '+'))
